In [ ]:
import copy

import ollama
import pandas as pd

from es.ct import PROMPT as PROMPT_CT


In [ ]:
model = "deepseek-r1"
dataset = "dwug-en"

In [ ]:
data = pd.read_csv("../test_dwug_en.csv")
data.shape

In [ ]:
data.columns

In [ ]:
PROMPTS = {
    "ct": PROMPT_CT,
}

In [ ]:
for prompt in PROMPTS.keys():
    predicted_score = []
    
    for _, row in data.iterrows():
        response = ollama.chat(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": PROMPTS[prompt].format(
                        target_word=row["lemma"],
                        sentence1=row["context_x"],
                        sentence2=row["context_y"],
                    ),
                }
            ],
            options={
                "max_tokens": 20,
            },
        )
        
        predicted_score.append(response["message"]["content"])
        
    df = pd.DataFrame()
    df["word"] = data["lemma"].tolist()
    df["identifier1"] = data["identifier1"].tolist()
    df["identifier2"] = data["identifier2"].tolist()
    df["grouping1"] = data["grouping_x"].tolist()
    df["grouping2"] = data["grouping_y"].tolist()
    df["judgment"] = data["judgment"].tolist()
    df["predicted_score"] = copy.deepcopy(predicted_score)
    df.to_csv(f"./{prompt}-{model}-{dataset}.csv", index=False)
    